In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=187829fe046e7827eaee4ed403a8b8941fbb3ea2520c909e84de48eed3732750
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark import SparkConf, SparkContext
from nltk import word_tokenize
from nltk.util import ngrams
from random import shuffle
import pyspark
import nltk
import numpy as np
import re
import os
import time

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sc = SparkContext(master="local", appName="BigDataLSH")

In [ ]:
newspapers = os.listdir('/content/drive/MyDrive/51800553_51800182_51800563_BigData_FinalProject/Dataset/newspapers/')
print(len(newspapers))

350


In [ ]:
data = sc.wholeTextFiles('/content/drive/MyDrive/51800553_51800182_51800563_BigData_FinalProject/Dataset/newspapers')
data.collect()[0]

('file:/content/drive/MyDrive/Big Data Final Project/Dataset/newspapers/Mạng đường sắt Việt Nam đến năm 2030.txt',
 'https://vnexpress.net/mang-duong-sat-viet-nam-den-nam-2030-4252544.html\r\nMạng đường sắt Việt Nam đến năm 2030\r\nMạng đường sắt Việt Nam đến năm 2030Ngành giao thông dự kiến hoàn thành xây mới đường sắt tốc độ cao Hà Nội - Vinh, Nha Trang - TP HCM đến năm 2030 và Vinh - Nha Trang đến 2050.')

**Text Cleaning**

In [ ]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.lower()
    text = re.sub('\W+', ' ', text)
    text = text.strip()
    
    return text

In [ ]:
data = data.map(lambda x: (x[0], x[1].split("\n")))
data.collect()[0]
data = data.map(lambda x: (x[0], clean_text(x[1][1]+x[1][2])))
data.collect()[0]

('file:/content/drive/MyDrive/Big Data Final Project/Dataset/newspapers/Mạng đường sắt Việt Nam đến năm 2030.txt',
 'mạng đường sắt việt nam đến năm 2030 mạng đường sắt việt nam đến năm 2030ngành giao thông dự kiến hoàn thành xây mới đường sắt tốc độ cao hà nội vinh nha trang tp hcm đến năm 2030 và vinh nha trang đến 2050')

**Bước 1: Tạo Shingling**

In [ ]:
def calculate_join_grams(text, n):
    words = word_tokenize(text)
    n_gram = ngrams(words, n)
    join_grams = [' '.join(list(gram)) for gram in n_gram]
    
    return join_grams

In [ ]:
start_time = time.time()

shingles = data.flatMap(lambda x: calculate_join_grams(x[1], 3))
shingles = shingles.distinct()
shingles = shingles.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(shingles))

--- 3.5054094791412354 seconds ---
132874


In [ ]:
def create_newspaper_vector(cleaned_title_and_content):   
    exist_booleans = list()
    for shingle in shingles:
        if cleaned_title_and_content.find(shingle) != -1:
            exist_booleans.append(1)
        else:
            exist_booleans.append(0)
            
    return exist_booleans

In [ ]:
start_time = time.time()

newspaper_vectors = data.map(lambda x: create_newspaper_vector(x[1]))
newspaper_vectors = newspaper_vectors.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(newspaper_vectors))

--- 107.01614809036255 seconds ---
350


**Bước 2: MinHashing**

In [ ]:
def create_permutation(number_shingles):
    random_permutation_numbers = [i for i in range(1, number_shingles + 1)]
    shuffle(random_permutation_numbers)
    return random_permutation_numbers

In [ ]:
start_time = time.time()

number_permutation = 100
number_shingles = len(shingles)

rdd_create_permutation = sc.parallelize([x for x in range(0, number_permutation)])
permutations = rdd_create_permutation.map(lambda x: create_permutation(number_shingles))

permutations = permutations.collect()


print(f'--- {time.time() - start_time} seconds ---')
print(len(permutations))

--- 10.268224239349365 seconds ---
100


In [ ]:
def create_signature_vector(newspaper_vector):
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
            
    return signature_vector

In [ ]:
start_time = time.time()

rdd_newspaper_vectors = sc.parallelize(newspaper_vectors)

signature_vectors = rdd_newspaper_vectors.map(lambda x: create_signature_vector(x))
signature_vectors = signature_vectors.collect()

print(f'--- {time.time() - start_time} seconds ---')
print(len(signature_vectors))

--- 204.2446644306183 seconds ---
350


**Bước 3: Thuật toán LSH**

In [ ]:
number_of_band = 20
number_of_row = int(len(permutations) / number_of_band)
print(number_of_band, number_of_row)

10 10


In [ ]:
start_time = time.time()

buckets = dict()

for index_newspaper, signature_vector in enumerate(signature_vectors):
    previous_row = 0
    for _ in range(number_of_band):
        key_chars = list()
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key not in buckets:
            buckets[key] = [index_newspaper]
        else:
            buckets[key].append(index_newspaper)

print(f'--- {time.time() - start_time} seconds ---')
print(buckets)
print(len(buckets))

--- 0.015986919403076172 seconds ---
{'3836_13213_14258_10178_1548_5826_6619_10209_297_11381': [0], '891_3241_2394_240_954_386_1144_534_293_7355': [0], '1952_4033_2887_208_212_7347_14233_1416_1225_1788': [0], '3333_1768_835_5691_2638_2775_14796_2201_370_3319': [0], '4637_2347_6328_15645_6671_4820_144_228_3979_3388': [0], '634_339_1100_1559_2909_431_8908_1572_5039_5166': [0], '759_3195_5129_1940_1822_2640_496_7691_3561_2080': [0], '9849_1274_2382_11361_4515_777_28_122_757_3565': [0], '5506_536_1165_3652_13315_1287_26_534_327_10796': [0], '1815_472_3536_13723_3608_5000_8031_3448_5600_6094': [0], '49_294_34_21_168_338_133_42_74_923': [1], '560_15_324_272_668_5_32_251_832_294': [1], '22_882_796_461_216_748_34_5_355_333': [1], '249_421_1179_29_212_465_76_416_341_96': [1], '23_223_381_342_392_322_468_21_1056_3': [1], '217_122_85_274_16_603_71_83_145_24': [1], '626_252_688_24_473_68_133_62_41_775': [1], '772_83_86_1513_357_58_225_195_420_283': [1], '122_615_39_102_148_426_49_452_1147_300': [1

In [ ]:
def create_bucket(signature_vector):
    buckets = dict()
    previous_row = 0
    for _ in range(number_of_band):
        key_chars = list()
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key not in buckets:
            buckets[key] = [index_newspaper]
        else:
            buckets[key].append(index_newspaper)
      
    return buckets


In [ ]:
start_time = time.time()

rdd_signature_vectors = sc.parallelize(signature_vectors)

dict_buckets = rdd_signature_vectors.map(lambda x: create_bucket(x))
dict_buckets = dict_buckets.collect()
print(dict_buckets)

buckets = dict()
for bucket in dict_buckets:
  buckets.update(bucket)

print(f'--- {time.time() - start_time} seconds ---')
print(len(buckets))

[{'3836_13213_14258_10178_1548_5826_6619_10209_297_11381': [349], '891_3241_2394_240_954_386_1144_534_293_7355': [349], '1952_4033_2887_208_212_7347_14233_1416_1225_1788': [349], '3333_1768_835_5691_2638_2775_14796_2201_370_3319': [349], '4637_2347_6328_15645_6671_4820_144_228_3979_3388': [349], '634_339_1100_1559_2909_431_8908_1572_5039_5166': [349], '759_3195_5129_1940_1822_2640_496_7691_3561_2080': [349], '9849_1274_2382_11361_4515_777_28_122_757_3565': [349], '5506_536_1165_3652_13315_1287_26_534_327_10796': [349], '1815_472_3536_13723_3608_5000_8031_3448_5600_6094': [349]}, {'49_294_34_21_168_338_133_42_74_923': [349], '560_15_324_272_668_5_32_251_832_294': [349], '22_882_796_461_216_748_34_5_355_333': [349], '249_421_1179_29_212_465_76_416_341_96': [349], '23_223_381_342_392_322_468_21_1056_3': [349], '217_122_85_274_16_603_71_83_145_24': [349], '626_252_688_24_473_68_133_62_41_775': [349], '772_83_86_1513_357_58_225_195_420_283': [349], '122_615_39_102_148_426_49_452_1147_300': 

In [ ]:
for _, bucket in buckets.items():
    if len(bucket) > 1:
        print(bucket)
        for index_newspaper in bucket:
            print(newspapers[index_newspaper])
        print()

**Bước 4: Test thuật toán bằng cách đưa vào một bản tin bất kỳ**

In [ ]:
def get_similar_newspapers(title_and_content, threshold=0.8):
    cleaned_title_and_content = clean_text(title_and_content)

    exist_booleans = list()
    for shingle in shingles:
        if shingle in cleaned_title_and_content:
            exist_booleans.append(True)
        else:
            exist_booleans.append(False)
    
    newspaper_vector = np.array(exist_booleans, dtype=bool)
    
    signature_vector = list()
    for permutation in permutations:
        min_number = float('inf')
        for index, exist_boolean in enumerate(newspaper_vector):
            if exist_boolean:
                if permutation[index] < min_number:
                    min_number = permutation[index]
        
        signature_vector.append(min_number)
    
    same_bucket_index_newspapers = set()
    
    key_chars = list()
    previous_row = 0
    for _ in range(number_of_band):
        for index in range(previous_row, previous_row + number_of_row):
            key_chars.append(signature_vector[index])
        previous_row += number_of_row

        key = '_'.join(map(str, key_chars))

        if key in buckets:
            same_bucket_index_newspapers.update(buckets[key])
        
    similar_index_newspapers = list()
    
    for same_bucket_index_newspaper in same_bucket_index_newspapers:
        count = 0
        for i, v in enumerate(newspaper_vector):
            if i > len(newspaper_vectors[same_bucket_index_newspaper]) - 1:
                break
                
            if newspaper_vectors[same_bucket_index_newspaper][i] == v:
                count += 1
        
        jaccard_similarity = count / len(newspaper_vectors[same_bucket_index_newspaper])

        if jaccard_similarity >= threshold:
            similar_index_newspapers.append(same_bucket_index_newspaper)
            
    return similar_index_newspapers


text = '''
Sáng 22/3, phiên họp thứ 4 Hội đồng bầu cử quốc gia diễn ra tại Nhà Quốc hội. Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia chủ trì phiên họp.Đến nay, Hội đồng bầu cử Quốc gia đã nhận được 1.136 hồ sơ người ứng cử ĐBQH khóa XV.Tổng Thư ký, Chủ nhiệm Văn phòng Quốc hội Nguyễn Hạnh Phúc, Chánh Văn phòng Hội đồng bầu cử quốc gia cho biết, thống kê sơ bộ, cho đến thời điểm này, Hội đồng bầu cử Quốc gia đã nhận được 1.136 hồ sơ người ứng cử ĐBQH khóa XV và 7.495 hồ sơ của người ứng cử đại biểu HĐND cấp tỉnh.Tính đến hết ngày 19/3, Ủy ban Mặt trận Tổ quốc các cấp đã hoàn thành việc tổ chức Hội nghị Hiệp thương lần thứ 2 để lập danh sách sơ bộ những người ứng cử ĐBQH và người ứng cử đại biểu Hội đồng nhân dân các cấp.Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia chủ trì phiên họp thứ 4 của Hội đồng.Như vậy, sau Hội nghị hiệp thương thứ hai, tổng số người ứng cử ĐBQH là 1.084 người, trong đó có 205 người do các cơ quan, tổ chức, đơn vị ở Trung ương giới thiệu, 803 người do các cơ quan, tổ chức, đơn vị ở địa phương giới thiệu, 76 người tự ứng cử, đạt tỉ lệ bình quân 2,17 lần so với tổng số ĐBQH được bầu...Khẩn trương triển khai các nhiệm vụ trong công tác bầu cửPhát biểu ý kiến kết luận phiên họp, Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia cho biết, Phiên họp thứ 4 Hội đồng bầu cử quốc gia đã hoàn thành các nội dung chương trình đề ra. Công tác chuẩn bị bầu cử đến nay trong phạm vi cả nước đã và đang được triển khai đúng pháp luật và bảo đảm theo tiến độ đề ra, chưa phát sinh vấn đề nào ảnh hưởng đến cuộc bầu cử.Năm đoàn kiểm tra đợt 1 của Hội đồng bầu cử quốc gia đánh giá các địa phương được kiểm tra đều rất trách nhiệm và thực hiện tốt những công việc theo đúng quy định hướng dẫn của Trung ương.Các cấp ủy đảng, chính quyền, Mặt trận Tổ quốc và các cơ quan, tổ chức, đoàn thể ở địa phương đã thực hiện rất tốt công tác chuẩn bị và đang khẩn trương nghiêm túc triển khai công việc chuẩn bị cho bầu cử.Tại Phiên họp thứ 4, Hội đồng bầu cử quốc gia đã thảo luận, tham gia góp ý những ý kiến về dự thảo báo cáo kết quả triển khai công tác bầu cử Quốc hội khóa XV và đại biểu HĐND các cấp nhiệm kỳ 2021-2026.Những ý kiến phát biểu tại Phiên họp thứ 4 tập trung vào các nội dung gồm: Xây dựng những kịch bản để phòng chống dịch và kiểm soát an toàn vệ sinh thực phẩm cũng như tổ chức bầu cử ở những nơi có cách ly y tế tập trung phòng, chống dịch Covid-19 thì cần tạo điều kiện thuận lợi, an toàn để cho người dân đi bỏ phiếu, kể cả những người đang được cách ly; về đối sách phòng chống các hoạt động xuyên tạc, phá hoại bầu cử; làm tốt công tác phòng chống cháy nổ trước và trong ngày bầu cử.Hội đồng bầu cử quốc gia thông qua nguyên tắc phân bổ đại biểu Trung ương về ứng cử ở địa phương cũng như rà soát, nhất trí với kết quả hiệp thương lần thứ hai. Chủ tịch Quốc hội đề nghị cơ quan chức năng tiếp tục chỉ đạo trong tình huống không chỉ có dịch bệnh, cần bổ sung khả năng về thiên tai có thể xảy ra vào thời điểm bầu cử trong tháng 5 tới. Các kịch bản phải chắc chắn, chặt chẽ.Theo Chủ tịch Quốc hội, việc chỉ đạo thành lập các tổ bầu cử, tập huấn bầu cử cũng như chỉ đạo giải quyết khiếu nại tố cáo đang được triển khai và sắp tới cần quan tâm hơn, nhất là khi đã xong Hội nghị hiệp thương lần thứ ba. Công tác này phải được thực hiện đúng quy định.Về việc bầu cử sớm, Chủ tịch Quốc hội đề nghị Văn phòng Hội đồng bầu cử quốc gia thông báo đến những địa phương có nhu cầu bầu cử sớm để đăng ký. Theo Luật Bầu cử ĐBQH và đại biểu Hội đồng nhân dân, việc bầu cử sớm do Hội đồng bầu cử quốc gia quyết định, khi bầu cử rồi mà bầu thiếu, muốn bầu cử lại hay không thì các địa phương sẽ báo cáo xin ý kiến Hội đồng bầu cử quốc gia và quyền cho bầu cử lại, bầu cử thêm cũng thuộc thẩm quyền của Hội đồng bầu cử quốc gia.Về quy định in ấn tài liệu, đấu thầu, Chủ tịch Quốc hội đề nghị Văn phòng Hội đồng bầu cử quốc gia có văn bản đề nghị Bộ Tài chính hướng dẫn bổ sung thêm trong trường hợp khẩn cấp cần in ấn không qua đấu thầu thì cũng cần có thêm có hướng dẫn.Chủ tịch Quốc hội cũng lưu ý một số công tác trong thực hiện về kê khai tài sản, hướng dẫn kê khai tài sản… cần được thực hiện theo đúng quy định của pháp luật.Về tuyên truyền, một số nơi có sáng kiến, nếu có sáng kiến tốt thì áp dụng rộng rãi cho cả nước. Chủ tịch Quốc hội đề nghị tăng cường tuyên truyền thêm, nhiều hơn, hiệu quả hơn cho đợt hội nghị hiệp thương lần thứ ba và cho tới ngày bầu cử 23/5 tới với mật độ tăng cường.Sau khi có kết quả hội nghị hiệp thương lần ba, sẽ lên danh sách chính thức những người ứng cử ĐBQH và Hội đồng nhân dân.Về bảo đảm cơ cấu ứng cử là nữ cho ĐBQH và Hội đồng nhân dân các cấp, hiện nay cố gắng thực hiện theo phương hướng bầu cử, tức là trong danh sách bầu cử có ít nhất 35% ứng cử viên là nữ. Chủ tịch Quốc hội cho rằng phải tiếp tục quan tâm tuyên truyền, nâng cao kiến thức, kỹ năng của các nữ ứng cử viên mà Trung ương Hội Liên hiệp Phụ nữ Việt Nam đã thực hiện rất tốt.Chủ tịch Quốc hội đề nghị Chánh Văn phòng Hội đồng bầu cử quốc gia tiếp thu đầy đủ các ý kiến của các thành viên Hội đồng bầu cử quốc gia để hoàn thiện các văn bản gửi tới các ĐBQH về báo cáo nhiệm kỳ, báo cáo của Hội đồng bầu cử quốc gia; đồng thời tập trung tham mưu giúp Hội đồng bầu cử quốc gia và các Tiểu ban đôn đốc theo dõi việc triển khai tiếp theo để chuẩn bị cho bầu cử.Các tiểu ban và Văn phòng của Hội đồng bầu cử quốc gia cũng như các cơ quan của Chính phủ, Ủy ban Trung ương Mặt trận Tổ quốc Việt Nam, các bộ, ban, ngành tiếp tục rà soát lại những công việc theo chức năng, nhiệm vụ được phân công để khẩn trương chỉ đạo, triển khai nhiệm vụ về công tác bầu cử...Tại phiên họp, Hội đồng bầu cử quốc gia đã xem xét thông qua hai nghị quyết đó là Nghị quyết về xử lý trường hợp khuyết người ứng cử Hội đồng nhân dân vì lý do bất khả kháng và Nghị quyết về nguyên tắc phân bổ ứng cử viên của Trung ương về địa phương ứng cử ĐBQH.Chủ tịch Quốc hội đề nghị Tiểu ban văn kiện, pháp luật và thông tin tuyên truyền, Tiểu ban nhân sự tiếp thu các ý kiến tại phiên họp để nhanh chóng hoàn thiện trình các văn bản. Riêng văn bản về nguyên tắc phân bổ đại biểu, đề nghị trình Chủ tịch Hội đồng bầu cử quốc gia sau khi được kiện toàn ký, để gắn với Hội nghị hiệp thương lần thứ ba...Chủ tịch Quốc hội Nguyễn Thị Kim Ngân, Chủ tịch Hội đồng bầu cử quốc gia nêu rõ, sau phiên họp, các cơ quan Chính phủ, Ủy ban Trung ương Mặt trận Tổ quốc Việt Nam, các bộ, ngành, địa phương và các cơ quan có liên quan tích cực chủ động trong công tác lãnh đạo, chỉ đạo. Việc nào cần hướng dẫn phải hướng dẫn sớm để cho các Ủy ban bầu cử biết, thực hiện và trả lời cho đúng. Theo TTXVN
'''

start_time = time.time()
similar_index_newspapers = get_similar_newspapers(text)
print(f'--- {time.time() - start_time} seconds ---')

print('Number of similar newspapers =', len(similar_index_newspapers))
for index_newspaper in similar_index_newspapers:
    print(newspapers[index_newspaper])

--- 1.7159383296966553 seconds ---
Number of similar newspapers = 1
Phó Chủ tịch Hà Nội mua vé, đi thử tàu Cát Linh - Hà Đông.txt
